# Pseudo Label Generation (Google Colab)

このノートブックは Google Colab 上で擬似ラベルを生成するためのテンプレートです。セルを 上から順番に実行してください。
- 必要に応じて Google Drive から `.mat` をダウンロード / マウントして `data/` に配置してください。
- 既存の `scripts/generate_pseudo_labels.py` と同じ処理をこの環境で再現します。
- GPU ランタイムを有効にしておくと計算が高速になります (Runtime -> Change runtime type -> GPU)。
- このリポジトリ一式を Colab 上で利用できるように (例: `!git clone ...` や `files.upload()` でアップロード) してから処理を実行してください。


In [ ]:
# Install required packages
!pip install --quiet torch torchvision gdown h5py pandas


In [ ]:
# Configure directories, shared Drive link, and download dataset if needed
import os
import subprocess
from pathlib import Path

SHARED_GDRIVE_URL = 'https://drive.google.com/file/d/1fa0gaEmbtGmqZ92L0EqzhH5LiMUAztix/view?usp=sharing'
DATA_DIR = Path('data')
ANNOTATIONS_DIR = Path('annotations')
DATA_DIR.mkdir(exist_ok=True)
ANNOTATIONS_DIR.mkdir(exist_ok=True)
DATASET_PATH = DATA_DIR / 'dataset.mat'
ANNOTATIONS_CSV = ANNOTATIONS_DIR / 'dataset_labels.csv'

# Default parameters used later in this notebook
IMAGE_KEY = 'Acq/Amp'
IMAGE_AXES = (0, 2, 1)
PERCENTILE = 20.0
SMOOTH_WINDOW = 5
MIN_STABLE_LENGTH = 3
LABEL_DTYPE = 'float32'

if not DATASET_PATH.exists():
    print(f'Downloading dataset to {DATASET_PATH} from Google Drive...')
    cmd = ['gdown', '--fuzzy', SHARED_GDRIVE_URL, '-O', str(DATASET_PATH)]
    result = subprocess.run(cmd, check=False)
    if result.returncode != 0:
        raise RuntimeError('Failed to download dataset from Google Drive. Please verify the link.')
else:
    print('Dataset already present:', DATASET_PATH)



In [ ]:
%%writefile generate_pseudo_labels.py
# Write pseudo label generation script
#!/usr/bin/env python3
"""Generate pseudo labels for ultrasound .mat sequences using full in-memory processing.

Frames are loaded entirely into memory, motion scores are computed in a vectorised way,
and results are exported as CSV with stability labels.
"""
from __future__ import annotations

import argparse
import csv
from pathlib import Path
from typing import Optional, Sequence, Tuple

import h5py
import numpy as np


def _interpret_image_shape(
    shape: Tuple[int, ...], override: Optional[Tuple[int, ...]] = None
) -> Tuple[int, int, int, int, Tuple[int, ...]]:
    if override is not None:
        axes = tuple(override)
        if len(axes) not in (3, 4):
            raise ValueError("image_axes override must have length 3 or 4")
        if len(axes) != len(shape):
            raise ValueError("image_axes override must match tensor rank")
        dims = [shape[a] for a in axes]
        if len(axes) == 4:
            n, c, h, w = dims
            return n, c, h, w, axes
        n, h, w = dims
        return n, 1, h, w, axes

    rank = len(shape)
    if rank == 4:
        candidates = [
            (0, 3, 1, 2),
            (0, 1, 2, 3),
            (3, 2, 0, 1),
            (3, 0, 1, 2),
        ]
        for axes in candidates:
            n = shape[axes[0]]
            c = shape[axes[1]]
            h = shape[axes[2]]
            w = shape[axes[3]]
            if all(x > 0 for x in (n, c, h, w)):
                return n, c, h, w, axes
        raise ValueError(f"Unable to infer N,C,H,W from shape {shape}")
    if rank == 3:
        candidates = [
            (0, 1, 2),
            (0, 2, 1),
            (2, 0, 1),
            (2, 1, 0),
            (1, 0, 2),
            (1, 2, 0),
        ]
        for axes in candidates:
            n = shape[axes[0]]
            h = shape[axes[1]]
            w = shape[axes[2]]
            if all(x > 0 for x in (n, h, w)):
                return n, 1, h, w, axes
        raise ValueError(f"Unable to infer N,H,W from shape {shape}")
    raise ValueError(f"Unsupported image rank {rank}; expected 3D/4D, got {shape}")


def _reorder_full_stack(
    dataset: h5py.Dataset,
    axes: Tuple[int, ...],
    dtype: str,
) -> np.ndarray:
    full = np.asarray(dataset, dtype=dtype)
    target_axes = tuple(range(len(axes)))
    reordered = np.moveaxis(full, axes, target_axes)
    frames = reordered.reshape(reordered.shape[0], -1)
    return frames


def _moving_average(arr: np.ndarray, window: int) -> np.ndarray:
    if window <= 1:
        return arr.astype(np.float64, copy=False)
    kernel = np.ones(window, dtype=np.float64) / float(window)
    padded = np.pad(arr, (window // 2,), mode="edge")
    smoothed = np.convolve(padded, kernel, mode="valid")
    if smoothed.shape[0] > arr.shape[0]:
        smoothed = smoothed[: arr.shape[0]]
    return smoothed.astype(np.float64, copy=False)


def compute_motion_scores(frames: np.ndarray) -> np.ndarray:
    diff = np.abs(frames[1:] - frames[:-1])
    scores = np.zeros(frames.shape[0], dtype=np.float64)
    if diff.size:
        scores[1:] = diff.mean(axis=1)
    return scores


def enforce_min_segment(labels: np.ndarray, min_len: int) -> np.ndarray:
    if min_len <= 1:
        return labels
    labels = labels.copy()
    run_start = None
    for idx, val in enumerate(labels):
        if val == 1 and run_start is None:
            run_start = idx
        elif val == 0 and run_start is not None:
            run_length = idx - run_start
            if run_length < min_len:
                labels[run_start:idx] = 0
            run_start = None
    if run_start is not None:
        run_length = len(labels) - run_start
        if run_length < min_len:
            labels[run_start:] = 0
    return labels


def save_csv(path: Path, scores: np.ndarray, smooth_scores: np.ndarray, labels: np.ndarray) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    with path.open("w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["frame", "label", "score", "smooth_score"])
        for idx, (lab, sc, sm) in enumerate(zip(labels, scores, smooth_scores)):
            writer.writerow([idx, int(lab), float(sc), float(sm)])


def parse_args() -> argparse.Namespace:
    parser = argparse.ArgumentParser(description="Pseudo-label ultrasound frames")
    parser.add_argument("--mat_path", type=str, required=True, help="Path to v7.3 .mat file")
    parser.add_argument("--image_key", type=str, default="Acq/Amp", help="Dataset key inside the .mat file")
    parser.add_argument("--image_axes", type=int, nargs='*', default=None, help="Axis override, e.g. 0 2 1")
    parser.add_argument("--output_csv", type=str, required=True, help="Destination CSV for labels")
    parser.add_argument("--smooth_window", type=int, default=5, help="Moving average window (1 to disable)")
    parser.add_argument("--percentile", type=float, default=20.0, help="Percentile threshold for stability")
    parser.add_argument("--min_stable_length", type=int, default=3, help="Minimum consecutive stable frames")
    parser.add_argument("--dtype", type=str, default="float32", help="Frame conversion dtype")
    return parser.parse_args()


def generate_labels(args: argparse.Namespace) -> None:
    image_axes = tuple(args.image_axes) if args.image_axes else None

    with h5py.File(args.mat_path, "r") as f:
        if args.image_key in f:
            dataset = f[args.image_key]
        else:
            found = next((k for k in f.keys() if k.endswith(args.image_key)), None)
            if found is None:
                raise KeyError(f"image_key '{args.image_key}' not found in {args.mat_path}")
            dataset = f[found]
        _, _, _, _, axes = _interpret_image_shape(dataset.shape, override=image_axes)
        frames = _reorder_full_stack(dataset, axes, args.dtype)

    scores = compute_motion_scores(frames)
    smooth_scores = _moving_average(scores, args.smooth_window)
    threshold = np.percentile(smooth_scores, args.percentile)
    labels = (smooth_scores <= threshold).astype(np.int32)
    labels = enforce_min_segment(labels, args.min_stable_length)
    save_csv(Path(args.output_csv), scores, smooth_scores, labels)


def main() -> None:
    args = parse_args()
    generate_labels(args)


if __name__ == "__main__":
    main()



In [ ]:
# Run pseudo label generation script and preview the first rows
import subprocess
import pandas as pd
from pathlib import Path

RESULT_CSV = ANNOTATIONS_CSV
cmd = [
    'python',
    'generate_pseudo_labels.py',
    '--mat_path', str(DATASET_PATH),
    '--image_key', IMAGE_KEY,
    '--image_axes', *[str(a) for a in IMAGE_AXES],
    '--output_csv', str(RESULT_CSV),
    '--percentile', str(PERCENTILE),
    '--smooth_window', str(SMOOTH_WINDOW),
    '--min_stable_length', str(MIN_STABLE_LENGTH),
    '--dtype', LABEL_DTYPE,
]
result = subprocess.run(cmd, check=False)
if result.returncode != 0:
    raise RuntimeError('Pseudo label generation failed. Please check earlier logs.')
display(pd.read_csv(RESULT_CSV).head())
